In [127]:
import pandas as pd
from scipy import stats
from functools import partial
from sklearn.metrics import auc
from sklearn.feature_selection import SelectFdr, chi2,f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
import numpy as np
from sklearn.feature_selection import f_classif,mutual_info_classif
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn.linear_model import LassoCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn import model_selection 
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [128]:
train_data=pd.read_csv("clsp/train.csv",header=[0])
print("Train Data Shape:",train_data.shape)
test_data=pd.read_csv("clsp/test.csv",header=[0])
print("Test Data Shape:",test_data.shape)

Train Data Shape: (279, 60485)
Test Data Shape: (71, 60484)


In [129]:
# train_data_store=train_data
# test_data_store=test_data
train_data=train_data_store
test_data=test_data_store

In [130]:
X=train_data.iloc[:,2:]
y=train_data.iloc[:,1]
X_test_final=test_data.iloc[:,1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)
print("Test data",X_test_final.shape)

X_train (251, 60483)
X_test (28, 60483)
y_train (251,)
y_test (28,)
Test data (71, 60483)


In [131]:
def lowVariance(X_train,X_test,X_test_final):
    constant_filter = VarianceThreshold(threshold=0.25)
    constant_filter.fit(X_train)
    X_train=constant_filter.transform(X_train)
    X_test=constant_filter.transform(X_test)
    X_test_final=constant_filter.transform(X_test_final)
    print("X_train",X_train.shape)
    print("X_test",X_test.shape)
    print("y_train",y_train.shape)
    print("y_test",y_test.shape)
    print("Test data",X_test_final.shape)
    return X_train,X_test,X_test_final
X_train,X_test,X_test_final=lowVariance(X_train,X_test,X_test_final)

X_train (251, 11773)
X_test (28, 11773)
y_train (251,)
y_test (28,)
Test data (71, 11773)


In [132]:
def dataframe(X_train,X_test,X_test_final,y_train,y_test):
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)
    X_test_final = pd.DataFrame(X_test_final)
    print("X_train",X_train.shape)
    print("X_test",X_test.shape)
    print("y_train",y_train.shape)
    print("y_test",y_test.shape)
    print("Test data",X_test_final.shape)
    X_train.index=range(len(X_train))
    y_train.index=range(len(y_train))
    X_test.index=range(len(X_test))
    y_test.index=range(len(y_test))
    X_test_final.index=range(len(X_test_final))
    return X_train,X_test,X_test_final,y_train,y_test
X_train,X_test,X_test_final,y_train,y_test=dataframe(X_train,X_test,X_test_final,y_train,y_test)

X_train (251, 11773)
X_test (28, 11773)
y_train (251, 1)
y_test (28, 1)
Test data (71, 11773)


In [133]:
def tTest(X_train,X_test,X_test_final,y_train):
    indexes1=list(np.where(y_train.Label == 1)[0])
    indexes2=list(np.where(y_train.Label == -1)[0])
    final_col=[]
    count=0
    for col in X_train.columns:
        print(count)
        count+=1
        values1=X_train.loc[indexes1,col]
        values2=X_train.loc[indexes2,col]
        _,p_value=stats.ttest_ind(values1,values2, equal_var = False)
        if(p_value>=0.01):
            final_col.append(col)
    X_train=X_train.drop(final_col, axis = 1)
    X_test=X_test.drop(final_col, axis = 1)
    X_test_final=X_test_final.drop(final_col, axis = 1)
    print("X_train",X_train.shape)
    print("X_test",X_test.shape)
    print("y_train",y_train.shape)
    print("y_test",y_test.shape)
    print("Test data",X_test_final.shape)
    return X_train,X_test,X_test_final

In [ ]:
X_train,X_test,X_test_final=tTest(X_train,X_test,X_test_final,y_train)

In [135]:
# X_train_temp=X_train
# X_test_temp=X_test
# X_test_final_temp=X_test_final

In [214]:
X_train=X_train_temp
X_test=X_test_temp
X_test_final=X_test_final_temp

In [215]:
X_train,X_test,X_test_final,y_train,y_test=dataframe(X_train,X_test,X_test_final,y_train,y_test)

X_train (251, 2960)
X_test (28, 2960)
y_train (251, 1)
y_test (28, 1)
Test data (71, 2960)


In [216]:
corr_matrix = X_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [217]:
X_train=X_train.drop(to_drop, axis = 1)
X_test=X_test.drop(to_drop, axis = 1)
X_test_final=X_test_final.drop(to_drop, axis = 1)

In [218]:
X_train,X_test,X_test_final,y_train,y_test=dataframe(X_train,X_test,X_test_final,y_train,y_test)

X_train (251, 2951)
X_test (28, 2951)
y_train (251, 1)
y_test (28, 1)
Test data (71, 2951)


In [219]:
X_train,X_test,X_test_final,y_train,y_test=dataframe(X_train,X_test,X_test_final,y_train,y_test)

X_train (251, 2951)
X_test (28, 2951)
y_train (251, 1)
y_test (28, 1)
Test data (71, 2951)


In [ ]:
roc_auc = []
count=0
for feature in X_train.columns:
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    #clf = GaussianNB()
    #clf=SVC(kernel='rbf')
    clf.fit(X_train[feature].to_frame(), y_train)
    y_pred = clf.predict(X_test[feature].to_frame())
    roc_auc.append(roc_auc_score(y_test, y_pred))
    print(count)
    count+=1
roc_values = pd.Series(roc_auc)
roc_values.index = X_train.columns
roc_values.sort_values(ascending =False, inplace = True)
sel = roc_values[roc_values>0.6]
print(len(sel))

In [438]:
# X_train_b = X_train
# X_test_b = X_test
# X_test_final_b=X_test_final
X_train = X_train_b
X_test= X_test_b
X_test_final=X_test_final_b

In [439]:
sel=roc_values[roc_values>0.6]
print(len(sel))

509


In [440]:
sel=sel.iloc[0:80]
print(len(sel))
X_train = X_train[sel.index]
X_test = X_test[sel.index]
X_test_final=X_test_final[sel.index]
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)
print("Test data",X_test_final.shape)

80
X_train (251, 80)
X_test (28, 80)
y_train (251, 1)
y_test (28, 1)
Test data (71, 80)


In [441]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB().fit(X_train,y_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
#print("Accuracy score (training): {0:.3f}".format(clf.roc_auc_score(X_train, y_train)))
print("Accuracy vlid:",metrics.roc_auc_score(y_train,y_pred_train))
print("Accuracy vlid:",metrics.roc_auc_score(y_test,y_pred_test))

Accuracy vlid: 0.667005334010668
Accuracy vlid: 0.6428571428571428


/Users/komal/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [442]:
y_pred_final=clf.predict(X_test_final)
data_res=pd.DataFrame()
data_res['id']=test_data['id']
data_res['Label']=y_pred_final
data_res.to_csv (r'sample.csv', index = False, header=True)